In [13]:
using Plots
import Plots: px
theme(:default)
ENV["LINES"] = 20

20

In [35]:
include("parser.jl")
include("models.jl")
download(SIGDataSet)

true

In [15]:
data  = map(parseDataLine, eachline("./data/Sigmorphon/task1/all/spanish-train-high"));
vocab = Vocabulary(data)
edata = encode(data,vocab);
test  = map(parseDataLine, eachline("./data/Sigmorphon/task1/all/spanish-test"));
tdata = encode(test,vocab);
dictionary = [parseDataLine(line) for line in  eachline("./data/unimorph/spa/spa") if line != ""];

In [16]:
vocab.chars.toElement;

In [17]:
vocab.specialIndices

(unk = 1, mask = 2, eow = 3, bow = 4)

In [18]:
trainsfs    = unique(map(x->join(x.surface),data));
testsfs     = unique(map(x->join(x.surface),test)); 
dictsfs     = unique([map(x->join(x.surface),dictionary); trainsfs; testsfs]);
unseensfs   = [x for x in dictsfs if x ∉ trainsfs]

314523-element Array{String,1}:
 "ababilladas"  
 "ababillada"   
 "ababillados"  
 "ababillado"   
 "ababillándose"
 "ababillaos"   
 "ababillarse"  
 "ababíllate"   
 ⋮              
 "zuzase"       
 "zuzasteis"    
 "zuzaste"      
 "zuzas"        
 "zuza"         
 "zuzo"         
 "zuzó"         

In [37]:
morph = EncAttentiveVAE(length(vocab.chars), edata.num; H=512, E=16, Z=8, concatz=true)
train_ae!(morph, data, vocab; optim=Adam(lr=0.002), epoch=40)

(loss = 25.152424862670898,)
(loss = 20.944303961181642,)
(loss = 19.777203521728516,)
(loss = 18.948151272583008,)
(loss = 18.09249168548584,)
(loss = 17.279334880065917,)
(loss = 16.511275077819825,)
(loss = 15.886029692077637,)
(loss = 15.354880126953125,)
(loss = 14.888697891235351,)
(loss = 14.466316050720215,)
(loss = 14.16699754486084,)
(loss = 13.86016085357666,)
(loss = 13.609251914978028,)
(loss = 13.348343374633789,)
(loss = 13.141375604248047,)
(loss = 12.976138705444336,)
(loss = 12.83149554901123,)
(loss = 12.677238688659669,)
(loss = 12.558918894958497,)
(loss = 12.446060032653808,)
(loss = 12.367303955078125,)
(loss = 12.301560801696777,)
(loss = 12.199133605957032,)
(loss = 12.177544729614258,)
(loss = 12.119846543884277,)
(loss = 12.04534634399414,)
(loss = 12.017223941040038,)
(loss = 11.953271673583984,)
(loss = 11.934177980041504,)
(loss = 11.89070922241211,)
(loss = 11.858240467834472,)
(loss = 11.80096760559082,)
(loss = 11.745944607543946,)
(loss = 11.7732846252

(encoder = LSTM{Param{KnetArray{Float32,3}}, Multiply{Param{KnetArray{Float32,2}}}}(input=38,hidden=512,embed=16), Wμ = Multiply{Param{KnetArray{Float32,2}}}(input=512 output=8), Wσ = Dense{ELU}(Linear(Multiply{Param{KnetArray{Float32,2}}}(input=512 output=8), Bias{Param{KnetArray{Float32,1}}}(length=8))), Weaμ = Linear(Multiply{Param{KnetArray{Float32,2}}}(input=512 output=512), Bias{Param{KnetArray{Float32,1}}}(length=512)), Weaσ = Linear(Multiply{Param{KnetArray{Float32,2}}}(input=512 output=512), Bias{Param{KnetArray{Float32,1}}}(length=512)), Wμa = Linear(Multiply{Param{KnetArray{Float32,2}}}(input=512 output=512), Bias{Param{KnetArray{Float32,1}}}(length=512)), Wσa = Linear(Multiply{Param{KnetArray{Float32,2}}}(input=512 output=512), Bias{Param{KnetArray{Float32,1}}}(length=512)), output = Linear(Multiply{Param{KnetArray{Float32,2}}}(input=512 output=38), Bias{Param{KnetArray{Float32,1}}}(length=38)), Wdec = Multiply{Param{KnetArray{Float32,2}}}(input=8 output=512), decoder = LST

In [38]:
morphv =  EncAttentiveVAE(length(vocab.chars), edata.num; H=512, E=16, Z=8, concatz=true)
transferto!(morphv.encoder, morph.encoder)
transferto!(morphv.Wμ, morph.Wμ)
transferto!(morphv.Wσa, morph.Wσa)
transferto!(morphv.Wμa, morph.Wμa)
transferto!(morphv.Wσ, morph.Wσ)
transferto!(morphv.Weaμ, morph.Weaμ)
transferto!(morphv.Weaσ, morph.Weaσ)
transferto!(morphv.dec_embed,morph.dec_embed)
train_vae!(morphv, data, vocab; optim=Adam(lr=0.002), epoch=40, kl_weight=0.0f0, kl_rate = 0.1f0, fb_rate=3)

(kl_weight = 0.1f0, fbr = 3, loss = 22.94505267944336)
(kl_weight = 0.2f0, fbr = 3, loss = 19.93977203979492)
(kl_weight = 0.3f0, fbr = 3, loss = 18.93708644104004)
(kl_weight = 0.4f0, fbr = 3, loss = 18.027853141784668)
(kl_weight = 0.5f0, fbr = 3, loss = 17.18918165283203)
(kl_weight = 0.6f0, fbr = 3, loss = 16.43978777770996)
(kl_weight = 0.70000005f0, fbr = 3, loss = 15.810721081542969)
(kl_weight = 0.8000001f0, fbr = 3, loss = 15.276646681213379)
(kl_weight = 0.9000001f0, fbr = 3, loss = 14.851471719360351)
(kl_weight = 1.0f0, fbr = 3, loss = 14.466966752624511)
(kl_weight = 1.0f0, fbr = 3, loss = 14.145613027954102)
(kl_weight = 1.0f0, fbr = 3, loss = 13.885805085754395)
(kl_weight = 1.0f0, fbr = 3, loss = 13.631450276184083)
(kl_weight = 1.0f0, fbr = 3, loss = 13.433302351379394)
(kl_weight = 1.0f0, fbr = 3, loss = 13.20930485534668)
(kl_weight = 1.0f0, fbr = 3, loss = 13.06755604095459)
(kl_weight = 1.0f0, fbr = 3, loss = 12.92245524597168)
(kl_weight = 1.0f0, fbr = 3, loss = 1

In [39]:
samples = sample(morphv, vocab, edata; N=10000);

In [40]:
unique(samples[findall([s ∈ trainsfs for s in samples])])

3290-element Array{String,1}:
 "desairarían"  
 "predichas"    
 "no añejes"    
 "aliaba"       
 "agregaras"    
 "abjurados"    
 "silenciara"   
 "obstruiréis"  
 ⋮              
 "afrentemos"   
 "enlataremos"  
 "atenazarás"   
 "descuajemos"  
 "despobláremos"
 "mire"         
 "implorabais"  

In [41]:
length(samples[findall([s ∈ trainsfs for s in samples])])

4716

In [42]:
unique(samples[findall([s ∈ testsfs for s in samples])])

12-element Array{String,1}:
 "nixtamalizare"
 "enrole"       
 "conviviese"   
 "explosionara" 
 "historiaba"   
 "reestructura" 
 "estragara"    
 "pinchare"     
 "sistematizaba"
 "balbuciera"   
 "puntaren"     
 "refundamos"   

In [43]:
length(samples[findall([s ∈ testsfs for s in samples])])

14

In [44]:
unique(length(samples[findall([s ∈ dictsfs for s in samples])]))

1-element Array{Int64,1}:
 6509

In [45]:
sampleinter(morphv, vocab, data; N=10)

12-element Array{String,1}:
 "abarrotare"
 "abarrotare"
 "abarrotare"
 "abarrotare"
 "abarrotare"
 "abarrotare"
 "abarrotare"
 "abarrotare"
 "abarrotare"
 "abarrotare"
 "abarrotare"
 "abarrotare"

In [46]:
au, sigma, mu = calc_au(morphv,tdata; delta=0.01)

(0, Float32[1.2510026e-22; 2.658151e-23; … ; 5.19158e-23; 1.1128319e-25], K32(8,1)[-4.2638976e-10⋯])

In [47]:
mi = calc_mi(morphv,tdata)

-0.036793243408203224

In [48]:
calc_ppl(morphv,tdata)

6.040073511622641

In [ ]:
atts = attentions(morph,edata, vocab);

In [ ]:
function getsingle(sfs,exs,perms, αs,i) 
    join(vocab.chars[sfs[i]]), map(e->join(vocab.chars[e]),exs[i][perms[i]]), map(x->x[:,perms[i],i],αs)
end

In [ ]:
i = rand(1:length(atts))
s1,e1,α1 =  getsingle(atts[i]...,rand(1:16))
ys = 1:512
h1 = heatmap(e1, ys, α1.αu; size=(1200,800), xtickfont = font(10, "Halvetica"), title= s1 * ", mu", xrotation = 45, titlefontsize=10, left_margin=100px, bottom_margin=100px, right_margin=100px);
h2 = heatmap(e1, ys, α1.ασ ;size=(1200,800), xtickfont = font(10, "Halvetica"), title=s1 * ", sigma", xrotation = 45, titlefontsize=10, left_margin=100px, bottom_margin=100px, right_margin=100px);

In [ ]:
println(s1);println(e1)

In [ ]:
h1

# To-Dos
1. Add unrelated surface forms to the examplars too see whether the attention mappings are meaningful
2. Comparing with Normal Vae
3. Different Sampling Function on LM ($top_k$, temperature)
4. Attention in Decoder
5. Attention to the hidden state sequence, not just to the final state.
6. Metric for testing generation quality?

In [235]:
include("models.jl")
include("parser.jl")

nsample_packed_sequence (generic function with 1 method)

In [236]:
morph3=nothing; morphv=nothing; morph=nothing; morph2=nothing; KnetLayers.gc();

In [237]:
morph2 = VAE(length(vocab.chars); H=512, E=16, Z=16, concatz=true)
train_ae!(morph2, data, vocab; optim=Adam(), B=16, epoch=20);

(loss = 24.226039373779297,)
(loss = 17.94939325866699,)
(loss = 14.63141618347168,)
(loss = 11.688678450012207,)
(loss = 9.555169277191162,)
(loss = 7.757795697784424,)
(loss = 6.42142120513916,)
(loss = 5.461640008163452,)
(loss = 4.630353671264649,)
(loss = 3.879816455459595,)
(loss = 3.2313405588150026,)
(loss = 2.7009236671447754,)
(loss = 2.2843154739379883,)
(loss = 1.964221185016632,)
(loss = 1.8177687578201294,)
(loss = 1.5614021615982057,)
(loss = 1.3386266157627105,)
(loss = 1.3330284670352937,)
(loss = 1.1747992154598237,)
(loss = 1.0711584807395935,)


In [222]:
length(vocab.chars)

38

In [210]:
morph3 = VAE(length(vocab.chars); H=512, E=16, Z=16)
transferto!(morph3.encoder, morph2.encoder)
transferto!(morph3.Wμ, morph2.Wμ)
transferto!(morph3.Wσ, morph2.Wσ)
transferto!(morph3.dec_embed, morph3.encoder.embedding)
train_vae!(morph3, data, vocab; B=16, optim=Adam(lr=0.002), epoch=40, kl_weight=0.0f0, kl_rate = 0.1f0, fb_rate=4)

(kl_weight = 0.1f0, fbr = 4, loss = 21.441761749267577)
(kl_weight = 0.2f0, fbr = 4, loss = 20.253545791625978)
(kl_weight = 0.3f0, fbr = 4, loss = 19.015092526245116)


InterruptException: InterruptException:

In [ ]:
samples = sample(morph3, vocab, edata; N=10000, useprior=true)

In [212]:
samples[findall([s ∈ trainsfs for s in samples])]

InterruptException: InterruptException:

In [213]:
samples[findall([s ∈ testsfs for s in samples])]

6-element Array{String,1}:
 "okupara"      
 "no enamore"   
 "situarías"    
 "vibrase"      
 "no deleguemos"
 "refundamos"   

In [214]:
samples[findall([s ∈ dictsfs for s in samples])]

InterruptException: InterruptException:

In [215]:
samples[findall([s ∈ unseensfs for s in samples])] 

InterruptException: InterruptException:

In [216]:
inters = sampleinter(morph3, vocab, data; N=10)

12-element Array{String,1}:
 "contrastemos"
 "contrastemos"
 "contrasteis" 
 "contrasteis" 
 "contrasteis" 
 "contrasteis" 
 "sobrecaríais"
 "respararíais"
 "recontaríais"
 "recontaríais"
 "recontaríais"
 "recontaríais"

In [217]:
findall([s ∈ dictsfs for s in inters])

6-element Array{Int64,1}:
  1
  2
  9
 10
 11
 12

In [218]:
au, sigma, mu = calc_au(morph3, tdata; delta=0.01)

(16, Float32[0.394202; 0.30743402; … ; 0.62694967; 0.25517163], K32(16,1)[0.03776209⋯])

In [219]:
mi =  calc_mi(morph3,tdata)

3.1721899108886724

In [ ]:
calc_ppl(morph3, edata; nsample=500, B=16)

In [138]:
include("models.jl")
include("parser.jl")

nsample_packed_sequence (generic function with 1 method)

In [ ]:
model = LSTM_LM(length(vocab.chars); H=768, E=16)

In [ ]:
train_rnnlm!(model, data, vocab; epoch=40, optim=Adam(), B=16)

In [ ]:
 calc_ppllm(model, tdata)

In [ ]:
samples = sample(model, vocab; N=10000)

In [ ]:
samples[findall([s ∈ trainsfs for s in samples])]

In [ ]:
samples[findall([s ∈ testsfs for s in samples])]

In [ ]:
samples[findall([s ∈ dictsfs for s in samples])]

In [ ]:
 calc_ppllm(model, edata)

In [202]:
?@eval

```
@eval [mod,] ex
```

Evaluate an expression with values interpolated into it using `eval`. If two arguments are provided, the first is the module to evaluate in.
